In [1]:
#using Revise 
#includet("../src/thermal_modeling.jl")
using Flux, DataFrames, CSV, ProgressMeter, Statistics
#using thermal_modeling: TNNCell

In [2]:
# Topology Definitions
mutable struct HeatTransferLayer{U,V,T}
    n_temps::Int
    n_targets::Int
    conductance_net::Dense{U,Matrix{V},Vector{T}}
    adj_mat::Matrix{Int8}
end

function HeatTransferLayer(n_input::Integer, n_temps::Integer, n_targets::Integer)
    # populate adjacency matrix
    adj_mat = zeros(Int8, n_temps, n_temps)
    k = 1
    for col_j in 1:n_temps
        for row_i in col_j + 1:n_temps
            adj_mat[row_i, col_j] = k
            k += 1
        end
    end
    adj_mat = adj_mat + adj_mat'
    n_conds = Int(0.5 * n_temps * (n_temps - 1))
    HeatTransferLayer(n_temps, n_targets,
                      Dense(n_input + n_targets, n_conds, σ),
                      adj_mat)
end

# overload struct to make it callable
function (m::HeatTransferLayer)(all_input)
    n_temps = m.n_temps
    prev_out = @view all_input[1:m.n_targets, :]
    temps = @view all_input[1:n_temps, :]
    
    conductances = m.conductance_net(all_input)
    
    # subtract, scale, and sum
    tmp = hcat([sum(temps[j, :] .- prev_out[i, :] .* conductances[m.adj_mat[i, j], :] 
                for j in 1:n_temps if j != i) 
                    for i in 1:m.n_targets]...)'
    # mutating arrays not allowed in zygote
    """tmp = zeros(eltype(prev_out), size(prev_out))
    for i in 1:m.n_targets
        for j in 1:n_temps
            if j != i
                @. tmp[i, :] += (temps[j, :] - prev_out[i, :]) * conductances[m.adj_mat[i, j], :]
            end
        end
    end"""


    return tmp
end

# specify what is trainable 
Flux.@functor HeatTransferLayer (conductance_net,)

mutable struct TNNCell{U <: Chain,V <: Real,S}
    sample_time::V
    ploss_net::U
    heat_net::HeatTransferLayer
    caps::Vector{V}
    prll::Parallel  # will be defined in inner constructor (no outer definition)
    state0::S
    function TNNCell(sample_time::V, ploss_net::U, heat_net::HeatTransferLayer, caps::Vector{V}, init_hidden::S) where {U <: Chain,V <: Real,S}
        new{U,V,S}(sample_time, ploss_net, heat_net, caps, Parallel(+, ploss_net, heat_net), init_hidden)
    end
end


function TNNCell(n_input::U, n_temps::U, n_targets::U, init_hidden::S) where {U <: Integer,S}
    ploss_net = Chain(Dense(n_input + n_targets, 8, σ),
                      Dense(8, n_targets, σ))
    heat_transfer = HeatTransferLayer(n_input, n_temps, n_targets)
    caps = 0.5f0 .* randn(Float32, n_targets) .- 3f0  # Gaussian mean=-3 std=0.5
    TNNCell(Float32(0.5), ploss_net, heat_transfer, caps, init_hidden)
end

function (m::TNNCell)(prev_̂y, x)
    x_non_temps, x_temps = x
    xx = vcat(prev_̂y, x_temps, x_non_temps)
    rh_ode = m.prll(xx)
    y = prev_̂y .+ m.sample_time .* 10f0.^m.caps .* rh_ode
    return y, prev_̂y
end

# specify what is trainable 
Flux.@functor TNNCell (ploss_net, heat_net, caps)

In [6]:
const n_input_temps = 2
const n_input_non_temps = 3
const n_total_inputs = n_input_temps + n_input_non_temps
const n_targets = 3
const n_temps = n_targets + n_input_temps
const n_profiles = 49

# smoke-test the topology
xs = [(rand(Float32, n_input_non_temps, n_profiles), 
        rand(Float32, n_input_temps, n_profiles)) for i in 1:10]
h = rand(Float32, n_targets, n_profiles)  # initial hidden state

m = Flux.Recur(TNNCell(n_input_non_temps+n_input_temps, n_temps, n_targets, h), h)

# predict
ys = [m(x) for x in xs]
ys[1]

3×49 Matrix{Float32}:
 0.700342  0.0512767  0.992513  0.191769  …  0.259375  0.222454  0.423781
 0.627296  0.207904   0.091095  0.775912     0.673337  0.44422   0.861122
 0.445868  0.999073   0.942948  0.198656     0.63479   0.660746  0.970222

In [7]:

ys = [rand(Float32, n_targets, n_profiles) for i in 1:10]
loss(x_l, y_l) = Statistics::mean(Flux.Losses.mse(m(x), y) for (x, y) in zip(x_l, y_l))


loss (generic function with 1 method)

In [3]:
function load_dataset()
    data = CSV.File("/home/wilhelmk/dev/projects/datasets/kaggle_emotor_temps.csv") |> DataFrame;
    # FE
    @. data[!, :i_norm] = sqrt(data.i_d^2 + data.i_q^2);
    data[!, :fe1] = data.i_norm / maximum(data.i_norm) .* data.motor_speed / maximum(data.motor_speed);
    data
end

struct ChunkedTensors

end


,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.450682,18.8052,19.0867,-0.350055,18.2932,0.00286557,0.00441914
2,-0.325737,18.8186,19.0924,-0.305803,18.2948,0.000256782,0.000605872
3,-0.440864,18.8288,19.0894,-0.372503,18.2941,0.00235497,0.00128959
4,-0.327026,18.8356,19.083,-0.316199,18.2925,0.00610467,2.55843e-5
5,-0.47115,18.857,19.0825,-0.332272,18.2914,0.00313282,-0.0643168
6,-0.538973,18.9015,19.0771,0.00914747,18.2906,0.00963612,-0.613635
7,-0.653148,18.9417,19.0746,0.23889,18.2925,0.00133701,-1.00565
8,-0.758392,18.9609,19.0825,0.395099,18.294,0.00142196,-1.28838
9,-0.727128,18.9735,19.0855,0.546623,18.292,0.000576553,-1.49053


In [5]:
describe(data)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,u_q,54.279,-25.2909,48.9382,133.037,0,Float64
2,coolant,36.23,10.6238,26.9001,101.599,0,Float64
3,stator_winding,66.3427,18.5858,65.1101,141.363,0,Float64
4,u_d,-25.1338,-131.53,-7.42975,131.47,0,Float64
5,stator_tooth,56.8786,18.134,56.0363,111.946,0,Float64
6,motor_speed,2202.08,-275.549,1999.98,6000.02,0,Float64
7,i_d,-68.7168,-278.004,-51.0938,0.0518967,0,Float64
8,i_q,37.4128,-293.427,15.774,301.708,0,Float64
9,pm,58.5068,20.857,60.2663,113.607,0,Float64


In [6]:
test_set_pids = [60, 62, 74]
target_cols = ["pm", "stator_tooth", "stator_winding", "stator_yoke"]
gdf = groupby(data, :profile_id)

,u_q,coolant,stator_winding,u_d,stator_tooth,motor_speed,i_d
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,4.92171,15.7175,19.7292,0.0511826,18.8894,99.9645,-0.000384623
2,4.94854,15.8908,19.7376,0.0748067,18.8958,100.018,-0.00200349
3,4.94148,15.9926,19.7407,0.0636479,18.9028,100.0,0.000156374
4,4.92118,16.1352,19.7275,0.0767381,18.9081,99.9803,0.0014655
5,4.92435,16.1654,19.7305,0.0845534,18.8761,100.023,0.000682373
6,4.93227,16.0686,19.7318,0.0287042,18.8553,100.003,0.000352695
7,4.93508,15.9281,19.7221,0.0442232,18.8559,99.9628,-4.42849e-5
8,4.95151,15.8281,19.7141,0.046617,18.8662,100.023,-0.0102887
9,4.7295,15.7325,19.7258,0.431713,18.8392,100.011,-0.575398


In [7]:
p_sizes = combine(gdf, nrow)
max_len_test = maximum(filter(:profile_id => n -> n in test_set_pids, p_sizes).nrow)
max_len_train = maximum(filter(:profile_id => n -> n ∉ test_set_pids, p_sizes).nrow)

n_test_profiles = length(test_set_pids)
n_train_profiles = length(keys(gdf)) - n_test_profiles
@show max_len_train, max_len_test
@show n_train_profiles, n_test_profiles;

(max_len_train, max_len_test) = (43971, 25600)
(n_train_profiles, n_test_profiles) = (66, 3)


In [8]:
c_input_temps = ["ambient", "coolant"]
c_temps = [target_cols..., c_input_temps...]
c_non_temps = [c for c in names(data) if c ∉ [c_temps..., "profile_id"]]

train_tensor_non_temp_x = zeros(Float32, (max_len_train, length(c_non_temps), n_train_profiles))
train_tensor_temps_x = zeros(Float32, (max_len_train, length(c_input_temps), n_train_profiles))
train_tensor_y = zeros(Float32, (max_len_train, length(target_cols), n_train_profiles))
train_sample_weights = zeros(Float32, (max_len_train, n_train_profiles))

test_tensor_non_temp_x = zeros(Float32, (max_len_test, length(c_non_temps), n_test_profiles))
test_tensor_temps_x = zeros(Float32, (max_len_test, length(c_input_temps), n_test_profiles))
test_tensor_y = zeros(Float32, (max_len_test, length(target_cols), n_test_profiles))
test_sample_weights = zeros(Float32, (max_len_test, n_test_profiles));


In [9]:
# fill in DataFrame information
test_p_idx = 0
train_p_idx = 0
@showprogress 0.5 "Computing " for (pid, df) in pairs(gdf)
    if pid.profile_id ∈ test_set_pids
        test_p_idx += 1
        test_tensor_non_temp_x[1:nrow(df), :, test_p_idx] .= df[:, c_non_temps]
        test_tensor_temps_x[1:nrow(df), :, test_p_idx] .= df[:, c_input_temps]
        test_tensor_y[1:nrow(df), :, test_p_idx] .= df[:, target_cols]
        test_sample_weights[1:nrow(df), test_p_idx] .= 1
    else
        train_p_idx += 1
        train_tensor_non_temp_x[1:nrow(df), :, train_p_idx] .= df[:, c_non_temps]
        train_tensor_temps_x[1:nrow(df), :, train_p_idx] .= df[:, c_input_temps]
        train_tensor_y[1:nrow(df), :, train_p_idx] .= df[:, target_cols]
        train_sample_weights[1:nrow(df), train_p_idx] .= 1
    end
end

Computing   3%|█▎                                       |  ETA: 0:02:11

Computing  17%|███████▏                                 |  ETA: 0:00:31

Computing  25%|██████████▏                              |  ETA: 0:00:23

Computing  29%|███████████▉                             |  ETA: 0:00:20

Computing  35%|██████████████▎                          |  ETA: 0:00:16

...

Computing  83%|█████████████████████████████████▉       |  ETA: 0:00:03

Computing  88%|████████████████████████████████████▎    |  ETA: 0:00:02

Computing  94%|██████████████████████████████████████▋  |  ETA: 0:00:01

Computing  99%|████████████████████████████████████████▍|  ETA: 0:00:00

Computing 100%|█████████████████████████████████████████| Time: 0:00:15


In [10]:
tbptt_len = 128

train_vec_temps_x = [train_tensor_temps_x[i, :, :] for i in 1:size(train_tensor_temps_x, 1)]
train_vec_non_temp_x = [train_tensor_non_temp_x[i, :, :] for i in 1:size(train_tensor_non_temp_x, 1)]
train_vec_x = collect(zip(train_vec_non_temp_x, train_vec_temps_x))
train_vec_y = [train_tensor_y[i, :, :] for i in 1:size(train_tensor_y, 1)]
train_vec_sample_weights = [train_sample_weights[i, :] for i in 1:size(train_sample_weights, 1)]

train_vec_chunked_x = []
train_vec_chunked_y = []
train_vec_chunked_w = []

i = 0;
while i*tbptt_len <= max_len_train
    push!(train_vec_chunked_x, train_vec_x[i*tbptt_len+1:minimum(((i+1)*tbptt_len+1, max_len_train))])
    push!(train_vec_chunked_y, train_vec_y[i*tbptt_len+1:minimum(((i+1)*tbptt_len+1, max_len_train))])
    push!(train_vec_chunked_w, train_vec_sample_weights[i*tbptt_len+1:minimum(((i+1)*tbptt_len+1, max_len_train))])
    global i+= 1
end

test_vec_temps_x = [test_tensor_temps_x[i, :, :] for i in 1:size(test_tensor_temps_x, 1)]
test_vec_non_temp_x = [test_tensor_non_temp_x[i, :, :] for i in 1:size(test_tensor_non_temp_x, 1)]
test_vec_x = collect(zip(train_vec_non_temp_x, test_vec_temps_x))
test_vec_y = [test_tensor_y[i, :, :] for i in 1:size(test_tensor_y, 1)]
test_vec_sample_weights = [test_sample_weights[i, :] for i in 1:size(test_sample_weights, 1)];


In [11]:
using Statistics: mean
n_epochs = 100
pbar = Progress(n_epochs, desc="Training Epochs", start=1, showspeed=true)
init_hidden = train_vec_y[1]
m = Flux.Recur(TNNCell(length(c_non_temps)+length(c_input_temps),
                       length(c_temps),
                       length(target_cols),
                       init_hidden),
               init_hidden)
ps = params(m)
opt = ADAM(1e-3)

function sample_weighted_loss(x::Vector{Tuple{Matrix{T}, Matrix{T}}}, y::Vector{U}, w::Vector{V}) where {T, U, V}
   mean(Flux.Losses.mse(m(xi), yi, agg=z->mean(wi' .* z)) for (xi, yi, wi) in zip(x, y, w)) 
end

sample_weighted_loss (generic function with 1 method)

In [22]:
# training
data_tup = zip(train_vec_chunked_x, train_vec_chunked_y, train_vec_chunked_w);
for epoch in 1:n_epochs
    Flux.reset!(m)
    Flux.train!(sample_weighted_loss, ps, data_tup, opt)
    next!(pbar, showvalues = [(:epoch, epoch)])
end



Training Epochs  2%|▌                     |  ETA: 8:39:30 ( 5.30  m/it)



Training Epochs  3%|▋                     |  ETA: 6:33:34 ( 4.06  m/it)



Training Epochs  4%|▉                     |  ETA: 5:49:30 ( 3.64  m/it)


In [18]:
using BenchmarkTools
data_tup = zip(train_vec_chunked_x, train_vec_chunked_y, train_vec_chunked_w);

In [19]:
@benchmark Flux.train!($sample_weighted_loss, $ps, $data_tup, $opt)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 167.081 s (6.47% GC) to evaluate,
 with a memory estimate of 51.98 GiB, over 353193566 allocations.

In [20]:
function train_one_epoch()
    Flux.train!($sample_weighted_loss, $ps, $data_tup, $opt)
end

ErrorException: syntax: "$" expression outside quote around /home/wilhelmk/dev/projects/hyperdyn/examples/thermal_modeling/notebooks/prototyping.ipynb:2

In [21]:
@benchmark train_one_epoch()

UndefVarError: UndefVarError: train_one_epoch not defined